## 18.1 视图
&emsp;&emsp;视图是虚拟的表。与包含数据的表不一样，视图只包含使用时动态检索数据的查询。

>**说明**：Microsoft Access 不支持视图，没有与SQL 视图一致的工作方式。因此，这一课的内容不适用Microsoft Access。MySQL 从版本5 起开始支持视图，因此，这一课的内容不适用较早版本的MySQL。SQLite 仅支持只读视图，所以视图可以创建，可以读，但其内容不能更改。

In [1]:
import sqlite3 as sql 
import pandas as pd

def select_sql(query):
    conn = sql.connect('tysql.sqlite')
    cur = conn.cursor()
    df = pd.read_sql(query, con=conn)
    cur.close()
    conn.close()
    print(df)

In [2]:
# 从三个表中检索数据
query = '''
SELECT cust_name, cust_contact
FROM Customers, Orders, OrderItems
WHERE Customers.cust_id = Orders.cust_id
        AND OrderItems.order_num = Orders.order_num
        AND prod_id = 'RGAN01';
'''
select_sql(query)

       cust_name        cust_contact
0        Fun4All  Denise L. Stephens
1  The Toy Store          Kim Howard


&emsp;&emsp;把整个查询包装成一个名为ProductCustomers 的虚拟表，则可以如下轻松地检索出相同的数据：

``` SQL
SELECT cust_name, cust_contact
FROM ProductCustomers
WHERE prod_id = 'RGAN01';
```

>**提示**：所有DBMS 非常一致地支持视图创建语法。

### 18.1.1 为什么使用视图
&emsp;&emsp;视图的一些常见应用：
- 重用SQL语句；
- 简化复杂的SQL 操作。在编写查询后，可以方便地重用它而不必知道其基本查询细节；
- 使用表的一部分而不是整个表；
- 保护数据。可以授予用户访问表的特定部分的权限，而不是整个表的访问权限；
- 更改数据格式和表示。视图可返回与底层表的表示和格式不同的数据。

&emsp;&emsp;视图仅仅是用来查看存储在别处数据的一种设施。视图本身不包含数据，因此返回的数据是从其他表中检索出来的。在添加或更改这些表中的数据时，视图将返回改变过的数据。

>**注意**：因为视图不包含数据，所以每次使用视图时，都必须处理查询执行时需要的所有检索。如果你用多个联结和过滤创建了复杂的视图或者嵌套了视图，性能可能会下降得很厉害。因此，在部署使用了大量视图的应用前，应该进行测试。

### 18.1.2 视图的规则和限制
&emsp;&emsp;视图创建和使用的一些最常见的规则和限制：
- 与表一样，视图必须唯一命名（不能给视图取与别的视图或表相同的名字）；
- 对于可以创建的视图数目没有限制；
- 创建视图，必须具有足够的访问权限。这些权限通常由数据库管理人员授予；
- 视图可以嵌套，即可以利用从其他视图中检索数据的查询来构造视图。所允许的嵌套层数在不同的DBMS 中有所不同（嵌套视图可能会严重降低查询的性能，因此在产品环境中使用之前，应该对其进行全面测试）；
- 许多DBMS 禁止在视图查询中使用ORDER BY 子句；
- 有些DBMS 要求对返回的所有列进行命名，如果列是计算字段，则需要使用别名；
- 视图不能索引，也不能有关联的触发器或默认值；
- 有些DBMS 把视图作为只读的查询，这表示可以从视图检索数据，但不能将数据写回底层表。详情请参阅具体的DBMS 文档；
- 有些DBMS 允许创建这样的视图，它不能进行导致行不再属于视图的插入或更新。

## 18.2 创建视图
&emsp;&emsp;视图用CREATE VIEW 语句来创建。

>**说明**：删除视图，可以使用DROP 语句，其语法为`DROP VIEW viewname;`，覆盖（或更新）视图，必须先删除它，然后再重新创建。

### 18.2.1 利用视图简化复杂的联结
&emsp;&emsp;一个最常见的视图应用是隐藏复杂的SQL，这通常涉及联结:

In [3]:
def commit_sql(query):
    conn = sql.connect('tysql.sqlite')
    cur = conn.cursor()
    cur.execute(query);
    cur.close()
    conn.commit()
    conn.close()
    print('Done!')

In [4]:
# 创建一个名为ProductCustomers 的视图，它联结三个表
# 返回已订购了任意产品的所有顾客的列表。
query = '''
CREATE VIEW ProductCustomers AS
SELECT cust_name, cust_contact, prod_id
FROM Customers, Orders, OrderItems
WHERE Customers.cust_id = Orders.cust_id
    AND OrderItems.order_num = Orders.order_num;
'''
commit_sql(query)

Done!


In [5]:
# 检索订购了产品RGAN01 的顾客
query = '''
SELECT cust_name, cust_contact
FROM ProductCustomers
WHERE prod_id = 'RGAN01';
'''
select_sql(query)

       cust_name        cust_contact
0        Fun4All  Denise L. Stephens
1  The Toy Store          Kim Howard


>**提示**：创建不绑定特定数据的视图是一种好办法。例如，上面创建的视图返回订购所有产品而不仅仅是RGAN01 的顾客（这个视图先创建）。扩展视图的范围不仅使得它能被重用，而且可能更有用。这样做不需要创建和维护多个类似视图。

### 18.2.2 用视图重新格式化检索出的数据
&emsp;&emsp;视图的另一常见用途是重新格式化检索出的数据：

In [6]:
# 在单个组合计算列中返回供应商名和位置
query = '''
SELECT RTRIM(vend_name) || ' (' || RTRIM(vend_country) || ')'
        AS vend_title
FROM Vendors
ORDER BY vend_name;
'''
select_sql(query)

                vend_title
0      Bear Emporium (USA)
1         Bears R Us (USA)
2    Doll House Inc. (USA)
3  Fun and Games (England)
4       Furball Inc. (USA)
5  Jouets et ours (France)


In [7]:
# 假设经常需要这个格式的结果，则把此语句转换为视图
query = '''
CREATE VIEW VendorLocations AS
SELECT RTRIM(vend_name) || ' (' || RTRIM(vend_country) || ')'
        AS vend_title
FROM Vendors;
'''
commit_sql(query)

Done!


In [8]:
# 查询视图检索数据
query = '''
SELECT *
FROM VendorLocations;
'''
select_sql(query)

                vend_title
0         Bears R Us (USA)
1      Bear Emporium (USA)
2    Doll House Inc. (USA)
3       Furball Inc. (USA)
4  Fun and Games (England)
5  Jouets et ours (France)


>**说明**：在这一课的前面提到，各种DBMS 中用来创建视图的语法相当一致。那么，为什么会有多种创建视图的语句版本呢？因为视图只包含一个SELECT 语句，而这个语句的语法必须遵循具体DBMS 的所有规则和约束，所以会有多个创建视图的语句版本。

### 18.2.3 用视图过滤不想要的数据
&emsp;&emsp;视图对于应用普通的WHERE 子句也很有用：

In [9]:
# 定义CustomerEMailList 视图，过滤没有电子邮件地址的顾客
query = '''
CREATE VIEW CustomerEMailList AS
SELECT cust_id, cust_name, cust_email
FROM Customers
WHERE cust_email IS NOT NULL;
'''
commit_sql(query)

Done!


In [10]:
# 查询视图检索数据
query = '''
SELECT *
FROM CustomerEMailList;
'''
select_sql(query)

      cust_id     cust_name             cust_email
0  1000000001  Village Toys  sales@villagetoys.com
1  1000000003       Fun4All     jjones@fun4all.com
2  1000000004       Fun4All  dstephens@fun4all.com
3  1000000006      Toy Land        sam@toyland.com


>**说明**：从视图检索数据时如果使用了一条WHERE 子句，则两组子句（一组在视图中，另一组是传递给视图的）将自动组合。

## 18.2.4 使用视图与计算字段
&emsp;&emsp;视图也可以用于简化计算字段：

In [13]:
# 创建视图检索某个订单中的物品，计算每种物品的总价格
query = '''
CREATE VIEW OrderItemsExpanded AS
SELECT order_num,
        prod_id,
        quantity,
        item_price,
        quantity*item_price AS expanded_price
FROM OrderItems;
'''
commit_sql(query)

Done!


In [14]:
# 查询视图检索数据
query = '''
SELECT *
FROM OrderItemsExpanded
WHERE order_num = 20008;
'''
select_sql(query)

   order_num prod_id  quantity  item_price  expanded_price
0      20008  RGAN01         5        4.99           24.95
1      20008    BR03         5       11.99           59.95
2      20008  BNBG01        10        3.49           34.90
3      20008  BNBG02        10        3.49           34.90
4      20008  BNBG03        10        3.49           34.90


## 18.3 小结
&emsp;&emsp;本章学习了视图，一种虚拟的表。它们包含的不是数据而是根据需要检索数据的查询。视图提供了一种封装SELECT 语句的层次，可用来简化数据处理，重新格式化或保护基础数据。